## Description：
这里跑通SDM的baseline

In [1]:
import os
import time
import pickle
import random
from datetime import datetime
from collections import defaultdict
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.preprocessing import LabelEncoder

import numpy as np
import pandas as pd

import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

# 从utils里面导入函数
from utils import get_data_set, gen_model_input, train_sdm_model
from utils import get_embeddings, get_sdm_recall_res

import warnings
warnings.filterwarnings('ignore')

DeepCTR version 0.9.0 detected. Your version is 0.8.2.
Use `pip install -U deepctr` to upgrade.Changelog: https://github.com/shenweichen/DeepCTR/releases/tag/v0.9.0


## 导入数据集

In [2]:
data_path = '../data_process'
data = pd.read_csv(os.path.join(data_path, 'train_data.csv'), index_col=0, parse_dates=['expo_time'])

In [3]:
# 选择出需要用到的列
use_cols = ['user_id', 'article_id', 'expo_time', 'net_status', 'exop_position', 'duration', 'city', 'age', 'gender', 'click', 'cat_1', 'cat_2']
data_new = data[use_cols]

## 划分数据集

In [4]:
# 按照用户分组，然后把最后一个item拿出来
click_df = data_new[data_new['click']==1]

In [5]:
def get_hist_and_last_click(all_click):
    all_click = all_click.sort_values(by=['user_id', 'expo_time'])
    click_last_df = all_click.groupby('user_id').tail(1)
    
    # 如果用户只有一个点击，hist为空了，会导致训练的时候这个用户不可见，此时默认泄露一下
    def hist_func(user_df):
        if len(user_df) == 1:
            return user_df
        else:
            return user_df[:-1]

    click_hist_df = all_click.groupby('user_id').apply(hist_func).reset_index(drop=True)

    return click_hist_df, click_last_df

In [6]:
user_click_hist_df, user_click_last_df = get_hist_and_last_click(click_df)

In [9]:
user_click_hist_df.head()

,user_id,article_id,expo_time,net_status,exop_position,duration,city,age,gender,click,cat_1,cat_2
0,17340,464481478,2021-06-30 20:34:47,2,21,27,上海,A_30_39,male,1,汽车,汽车/买车
1,17340,465148736,2021-07-02 19:35:03,5,23,49,上海,A_30_39,male,1,旅游,旅游/旅游资讯
2,17340,464707540,2021-07-02 19:47:06,5,25,174,上海,A_30_39,male,1,健康,健康/疾病防护治疗及西医用药
3,17340,464993414,2021-07-02 19:47:06,5,27,11,上海,A_30_39,male,1,搞笑,搞笑/段子
4,17340,465115022,2021-07-02 20:01:34,5,41,14,上海,A_30_39,male,1,房产,房产/买房卖房


## SDM召回

In [7]:
def sdm_recall(data, topk=200, embedding_dim=32, SEQ_LEN_short=5, SEQ_LEN_prefer=50,
                      batch_size=64, epochs=1, verbose=1, validation_split=0.0):
    """通过SDM模型，计算用户向量和文章向量
    param: data: 用户日志数据
    topk: 对于每个用户，召回多少篇文章
    """
    user_id_raw = data[['user_id']].drop_duplicates('user_id')
    doc_id_raw = data[['article_id']].drop_duplicates('article_id')
    
    # 类别数据编码   
    base_features = ['user_id', 'article_id', 'city', 'age', 'gender', 'cat_1', 'cat_2']
    feature_max_idx = {}
    for f in base_features:
        lbe = LabelEncoder()
        data[f] = lbe.fit_transform(data[f]) + 1
        feature_max_idx[f] = data[f].max() + 1
        
    # 构建用户id词典和doc的id词典，方便从用户idx找到原始的id
    user_id_enc = data[['user_id']].drop_duplicates('user_id')
    doc_id_enc = data[['article_id']].drop_duplicates('article_id')
    user_idx_2_rawid = dict(zip(user_id_enc['user_id'], user_id_raw['user_id']))
    doc_idx_2_rawid = dict(zip(doc_id_enc['article_id'], doc_id_raw['article_id']))
    
    user_profile = data[['user_id', 'gender', 'age', 'city']].drop_duplicates('user_id')
    item_profile = data[['article_id']].drop_duplicates('article_id')
    user_profile.set_index('user_id', inplace=True)
    
    train_set, test_set = get_data_set(user_click_hist_df, seq_short_len=SEQ_LEN_short, seq_prefer_len=SEQ_LEN_prefer)
    
    train_model_input, train_label = gen_model_input(train_set, user_profile, SEQ_LEN_short, SEQ_LEN_prefer)
    test_model_input, test_label = gen_model_input(test_set, user_profile, SEQ_LEN_short, SEQ_LEN_prefer)
       
    # 构建模型并完成训练
    model = train_sdm_model(train_model_input, train_label, embedding_dim, feature_max_idx, SEQ_LEN_short, SEQ_LEN_prefer, batch_size, epochs, verbose, validation_split)
    
    # 获得用户embedding和doc的embedding， 并进行保存
    user_embs, doc_embs = get_embeddings(model, test_model_input, user_idx_2_rawid, doc_idx_2_rawid)
    
    # 对每个用户，拿到召回结果并返回回来
    user_recall_items_dict = get_sdm_recall_res(user_embs, doc_embs, user_idx_2_rawid, doc_idx_2_rawid, topk)
    
    return user_recall_items_dict

In [8]:
user_recall_doc_dict = sdm_recall(user_click_hist_df)

100%|██████████████████████████████████████████████████████████████████████████| 20000/20000 [00:08<00:00, 2338.31it/s]


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.


...

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Train on 526056 samples
526056/526056 [==============================] - 1315s 3ms/sample - loss: 0.3605


In [9]:
len(user_recall_doc_dict)

19999

## 评估

In [10]:
# 依次评估召回的前10, 20, 30, 40, 50个文章中的击中率
def metrics_recall(user_recall_items_dict, trn_last_click_df, topk=100):
    last_click_item_dict = dict(zip(trn_last_click_df['user_id'], trn_last_click_df['article_id']))
    user_num = len(user_recall_items_dict)
    
    for k in range(50, topk+1, 50):
        hit_num = 0
        for user, item_list in user_recall_items_dict.items():
            if user in last_click_item_dict:
                # 获取前k个召回的结果
                tmp_recall_items = [x[0] for x in user_recall_items_dict[user][:k]]
                if last_click_item_dict[user] in set(tmp_recall_items):
                    hit_num += 1
        
        hit_rate = round(hit_num * 1.0 / user_num, 5)
        print(' topk: ', k, ' : ', 'hit_num: ', hit_num, 'hit_rate: ', hit_rate, 'user_num : ', user_num)

In [12]:
metrics_recall(user_recall_doc_dict, user_click_last_df, topk=200)

 topk:  50  :  hit_num:  28 hit_rate:  0.0014 user_num :  19999
 topk:  100  :  hit_num:  51 hit_rate:  0.00255 user_num :  19999
 topk:  150  :  hit_num:  70 hit_rate:  0.0035 user_num :  19999
 topk:  200  :  hit_num:  90 hit_rate:  0.0045 user_num :  19999
